<a href="https://colab.research.google.com/github/gecitemre/ai-codereviewer/blob/main/CENG403_2025_Spring_THE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Take-Home Exam 2 (THE-2)**
## CENG403 - Spring 2025

In this THE, we will focus on implementing a CNN and an RNN from scratch or using PyTorch:

*   Task 1: Implementing a forward function of your own deformable CNN from scratch (30 pts).
*   Task 2: Implementing CNN in PyTorch (40 pts).
*   Task 3: Implementing RNN using torch.autograd (30 pts).

**Getting Ready**

You can use the following tutorials if you need more details about the libraries/tools you will use:

*   **Jupyter Notebook and Colab**:
 * https://www.dataquest.io/blog/jupyter-notebook-tutorial/
 * https://colab.research.google.com/
* **NumPy**
 * https://numpy.org/devdocs/user/quickstart.html
* **PyTorch**:
 * https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html


## **Task 1: Implement a Forward Pass of Your Own Deformable CNN (30 Points)**

In this task, you are responsible for implementing the forward pass of [deformable convolution v2](https://arxiv.org/abs/1811.11168).

**Note that you should implement all functions from scratch! Using PyTorch or any other libraries except for `numpy` in your implementation will be evaluated as 0 (zero).**


### **1.1 Forward Pass Implementation (30 Points)**

Standard convolution (along one dimension of the input layer) with a filter size of $K$ at location $\mathbf{p}_0$ can be formally defined as:

$$
\mathbf{a}^{l+1}(\mathbf{p}_0) = \sum_{k=0}^{K-1}  \mathbf{w}(k) \cdot \mathbf{a}^l(\mathbf{p}_0 + \mathbf{p}_k),
$$

where:

*    $\mathbf{a}^l$ is the input feature map,
*    $\mathbf{a}^{l+1}$ is the output feature map,
*    $\mathbf{w}$ is the convolution kernel,
*    $\mathbf{p}_k$ is the fixed offset of the $k$-th sampling location (e.g., for a 1D feature map and a filter size of 3, $\mathbf{p}_k \in \{0, 1, 2\}$).


Deformable convolution v2 introduces learnable offsets $\Delta \mathbf{p}_k$ and modulation scalars $m_k \in [0,1]$:

$$
\mathbf{a}^{l+1}(\mathbf{p}_0) = \sum_{k=0}^{K-1} \mathbf{w}(k) \cdot \left( m_k \cdot \mathbf{a}^l(\mathbf{p}_0 + \mathbf{p}_k + \Delta \mathbf{p}_k) \right),
$$

where:

*    $\Delta \mathbf{p}_k$ are learned dynamic offsets,
*    $m_k$ are learned modulation scalars (also called a mask).

Since $\mathbf{p}_0 + \mathbf{p}_k + \Delta \mathbf{p}_k$ may be fractional, bilinear interpolation is used. For any fractional position $\mathbf{q}$ with $x$ and $y$ coordinates  $\mathbf{q} = (q_x, q_y)$, we first identify the closest integer positions:

$$
\begin{aligned}
\mathbf{p}_{\text{lt}} &= (\lfloor q_x \rfloor, \lfloor q_y \rfloor) \quad \text{(left top)}, \\
\mathbf{p}_{\text{rt}} &= (\lceil q_x \rceil, \lfloor q_y \rfloor) \quad \text{(right top)}, \\
\mathbf{p}_{\text{lb}} &= (\lfloor q_x \rfloor, \lceil q_y \rceil) \quad \text{(left bottom)}, \\
\mathbf{p}_{\text{rb}} &= (\lceil q_x \rceil, \lceil q_y \rceil) \quad \text{(right bottom)},
\end{aligned}
$$

where $\lfloor \cdot \rfloor$ denotes rounding down, and $\lceil \cdot \rceil$ rounding up to the closest integer.

Given these closest integer positions, bilinear interpolation for a fractional position $\mathbf{q}$ can be defined as:

$$
\mathbf{a}(\mathbf{q}) = \sum_{\mathbf{p}_i \in \{\mathbf{p}_{\text{lt}}, \mathbf{p}_{\text{rt}}, \mathbf{p}_{\text{lb}}, \mathbf{p}_{\text{rb}}\}} G(\mathbf{p}_i, \mathbf{q}) \cdot \mathbf{a}(\mathbf{p}_i),
$$

where the bilinear interpolation weight $G(\mathbf{p}, \mathbf{q})$ is:

$$
G(\mathbf{p}, \mathbf{q}) = (1 - |{p}_x - {q}_{x}|) \cdot (1 - |{p}_y - {q}_{y}|).
$$

With these definitions, we can write down deformable convolution in a complete form as follows:

$$
\mathbf{a}^{l+1}(\mathbf{p}_0) = \sum_{k=0}^{K-1} \mathbf{w}(k) \cdot m_k \cdot \left( \sum_{\mathbf{p}_i} G(\mathbf{p}_i, \mathbf{p}_0 + \mathbf{p}_k + \Delta \mathbf{p}_k) \cdot \mathbf{a}^l(\mathbf{p}_i) \right).
$$

For more details, please check the [paper](https://arxiv.org/abs/1811.11168) or [this tutorial](https://pub.towardsai.net/review-dcnv2-deformable-convnets-v2-object-detection-instance-segmentation-3d8a18bee2f5).

You are expected to complete the given `deformable_conv2d_np()` function.

**Do not change functions arguments and return value.**

In [ ]:
import numpy as np

def bilinear_interpolate(a_l, q_y, q_x):
    """
    Perform bilinear interpolation on the input activation map at the given (fractional) coordinates.

    Args:
        a_l (np.ndarray): 2D array of shape (H, W) representing the activation map (feature map) at a certain layer.
        q_y (float): Y-coordinate (row index) where interpolation is to be performed.
        q_x (float): X-coordinate (column index) where interpolation is to be performed.

    Returns:
        out (np.ndarray): Interpolated value at (q_x, q_y).
    """

    pass

    return out

def deform_conv2d_np(a_l, delta, mask, weight, stride=1, padding=0, dilation=1):
    """
    Deformable Conv2D v2 operation (forward pass) implemented in NumPy.

    Args:
        a_l (np.ndarray): Input feature map of shape (N, C_in, H_in, W_in),
                            where N is the batch size, C_in is the number of
                            input channels, and (H_in, W_in) are the height and
                            width of the input feature map. input corresponds to
                            'a^l' in the above formulation.

        delta (np.ndarray): Learned/estimated offsets of shape (N, 2 * K_h * K_w, H_out, W_out),
                             where K_h and K_w are the kernel height and width,
                             and (H_out, W_out) are the spatial dimensions of
                             the output feature map. The offset tensor corresponds
                             to 'Delta-p' in the above formulation and provides
                             the x and y displacements for each sampled point.

        mask (np.ndarray): Learned modulation masks of shape (N, K_h*K_w, H_out, W_out).
                           Corresponds to 'm' in the above formulation.

        weight (np.ndarray): Convolution kernel of shape (C_out, C_in, K_h, K_w),
                             where C_out is the number of output channels. Corresponds
                             to 'w' in the above formulation.

        stride (int): Stride of the convolution. Determines the spacing between
                               sampled input locations. Default is 1.

        padding (int): Zero-padding added to both sides of the input along height and width.
                                Default is 0.

        dilation (int): Dilation factor for the convolution kernel.
                                 Controls the spacing between kernel elements. Default is 1.

    Returns:
        out (np.ndarray): Output feature map of shape (N, C_out, H_out, W_out),
                          where each position is computed via deformable convolution using
                          bilinearly interpolated input values and learned offsets.
                          Corresponds to 'a^l+1' in the above formulation.
    """
    #########################################################
    # @TODO: Step 1: Preparing hyperparameters, pad input, and initialize output
    #########################################################
    pass


    #########################################################
    # @TODO: Step 2: Iterating over all coordinates
    # HINT: You should constuct nested loops considering all dimensions of input and kernel
    #########################################################
    pass


    #########################################################
    # @TODO: Step 3: Calculating delta (offset) and mask
    # HINT: Delta contains (dy, dx) for each kernel location; mask contains modulation strength.
    #########################################################
    pass

    #########################################################
    # @TODO: Step 4: Computing the deformed sampling position
    # HINT: Compute the deformed sampling position
    #########################################################
    pass


    #########################################################
    # @TODO: Step 5: Bilinear interpolation from the input feature map
    # HINT: Use bilinear interpolation to sample nearby pixel values.
    #########################################################
    pass

    #########################################################
    # @TODO: Step 6: Applying the convolution weight and modulation mask
    # HINT: Multiply interpolated value by the learned weight and the modulation scalar.
    #########################################################
    pass

    return out

### **1.2 Validate Implementations in Sections 1.1**

Run the following cell to validate/check whether your implementations in Sections 1.1 are correct. You will see your grade calculated for this part.

**Do not change/add any code here.**

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.ops import deform_conv2d
np.random.seed(0)
torch.manual_seed(0)

# --- Define parameters ---
N, C_in, H_in, W_in = 1, 3, 5, 5
C_out, K_h, K_w = 2, 3, 3
stride, padding, dilation = 1, 0, 1

# Input
np_input = np.arange(N * C_in * H_in * W_in, dtype=np.float32).reshape(N, C_in, H_in, W_in)
torch_input = torch.tensor(np_input, dtype=torch.float32)

# Offset: zero (no deformation)
np_offset = np.random.rand(N, 2 * K_h * K_w, 3, 3).astype(np.float32)
torch_offset = torch.tensor(np.copy(np_offset), dtype=torch.float32)

# Mask: ones (no modulation)
np_mask = np.ones((N, K_h * K_w, 3, 3), dtype=np.float32)
torch_mask = torch.tensor(np_mask, dtype=torch.float32)

# Weight: center=1
np_weight = np.random.rand(C_out, C_in, K_h, K_w).astype(np.float32)
torch_weight = torch.tensor(np.copy(np_weight), dtype=torch.float32)

# Bias: zero
np_bias = np.zeros((C_out), dtype=np.float32)
torch_bias = torch.tensor(np.copy(np_bias), dtype=torch.float32)

# --- Run NumPy Deformable Conv ---
np_output = deform_conv2d_np(np_input, np_offset, np_mask, np_weight,
                             stride=stride, padding=padding, dilation=dilation)

# --- Run PyTorch Deformable Conv ---
torch_output = deform_conv2d(
    input=torch_input,
    offset=torch_offset,
    weight=torch_weight,
    bias=torch_bias,
    stride=stride,
    padding=padding,
    dilation=dilation,
    mask=torch_mask
)

# --- Compare outputs ---
print("NumPy Output:\n", np_output[0, 0])
print("PyTorch Output:\n", torch_output[0, 0].detach().numpy())

grade = 0
if (np.allclose(np_output, torch_output.detach().numpy(), atol=1e-4) == True):
    grade = 30
else:
    grade = 0
print(" Your grade is ", grade, "/30.")

## **Task 2. Implement a CNN in PyTorch (40 pts)**

In this task, you are expected to implement a training, validation and testing pipeline for a CNN model using PyTorch. You will work with CIFAR100 dataset which is available in torchvision.dataset.


### **2.1 Implementing DataLoader (2.5 Points)**
In this task, you should implement a dataloader for CIFAR100 dataset. It is  a built-in dataset in the torchvision.datasets module, i.e. **you do not need to implement custom dataloader**.  Here, you can find examples about how a built-in dataset can be used:

*   https://pytorch.org/vision/main/datasets.html

The official CIFAR100 dataset contains only training and test splits. Therefore, you are expected to create a validation split from training data randomly, with **80% used for training** and **the remaining 20% reserved for validation**.


In [ ]:
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms

# Create DataLoader
batch_size = None # set your batch size

# Define transformations
transform_train = transforms.Compose([

   #####################################################
    # @TODO: You can add your data augmentations here
    # HINT: Do not forget to convert images to tensor
    #####################################################
])

transform_validationAndtest = transforms.Compose([

    #####################################################
    # HINT: Do not forget to convert images to tensor
    #####################################################
])


#####################################################
# @TODO:Define full training set
#####################################################
full_train_set = None # Modify this line


#####################################################
# @TODO: Calculate split lengths , Perform split
#####################################################
train_size = None  # Modify this line
val_size = None  # Modify this line

train_set, val_set = None, None # @TODO: Modify this line

# Update val_set transform to use validation transform
val_set.dataset.transform = transform_validationAndtest

#####################################################
# @TODO: Define Data loaders for training and validation splits
#####################################################
train_loader = None # Modify this line
val_loader = None # Modify this line

#####################################################
# @TODO: Define Test set and loader.
#####################################################
test_set = None # Modify this line
test_loader = None #  Modify this line

# ================================
# Print Split Ratios
# ================================
print(f"Total samples in CIFAR-100 training set: {len(full_train_set)}")
print(f"Training split: {len(train_set)} samples ({len(train_set)/len(full_train_set)*100:.2f}%)")
print(f"Validation split: {len(val_set)} samples ({len(val_set)/len(full_train_set)*100:.2f}%)")
print(f"Test split: {len(test_set)} samples")

### **2.2 Implement a CNN Model (2.5 Points)**
In this task, you are expected to implement CustomCNN class according to the following specifications:

*   Use at least three Conv2d Layers,
*   Use at least one Pooling2d layer,
*   Use at least two fully-connected layers,
*   Do not use any normalization layer.

There are no limitations for the model as long as you follow the above specifications.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, norm_layer= None):
        super(CustomCNN, self).__init__()

        #####################################################
        # @TODO: Define layers of CNN. Do not forget to follow model specifications!.
        pass
        #####################################################


    def forward(self, x):

        #####################################################
        # @TODO: Implement the forward pass
        # HINT: Apply activation functions like ReLU where necessary
        pass
        #####################################################

        return x


# Create model instance
model = CustomCNN()

# Print model summary
print(model)


###**2.3 Define Your Optimizer and Loss function**
This task involves setting up a loss function and an optimizer for training a model. The loss function measures prediction accuracy. The optimizer, such as SGD or Adam, updates model weights based on the loss. Key parameters like learning rate and weight decay should be set appropriately. The model must also be moved to the correct device (CPU/GPU) before training.

Here, you can reuse the relevant bit of your solution to THE1.


In [ ]:
import torch.optim as optim

#####################################################
# @TODO: Define a loss function
loss_function = None  # Replace with appropriate loss function for our classification task
#####################################################

#####################################################
# @TODO: Set your learning_rate, weight_decay or other optimizer-related parameters.
pass
#####################################################

#####################################################
# @TODO: Replace with your model instance
model = CustomCNN()
#####################################################

#####################################################
# @TODO: pass model to device (CPU or GPU)
pass
#####################################################

#####################################################
# @TODO: Choose an optimizer
# HINT: Use optim.SGD or optim.Adam
optimizer = None # Modify this line
#####################################################

# Print optimizer and loss function
print("Loss Function:", loss_function)
print("Optimizer:", optimizer)

### **2.4 Optional Steps**
You can use here for additional codes/implementations if necessary.

### **2.5 Implement Training Pipeline (10 Points)**

This task involves implementing training and validation loops for a model.

* The `train()` function:
 * Iterates through the training data (`train_loader`).
 * For each batch, extracts images and labels, moves them to the appropriate device (CPU/GPU), performs a forward pass (calculates predictions and loss), and performs the backward pass (computes gradients and updates the model weights).
 * Returns the average training loss, top-1 and top-5 accuracies. (Top-1 accuracy measures how often the class with the highest prediction scores matches the ground truth, while Top-5 accuracy checks if the ground truth label is among the top 5 classes with the highest prediction scores.)

* The `validate()` function:
 * Iterates through the validation data (`val_loader`) without updating the model weights (using `torch.no_grad()`).
 * Computes predictions, loss, and calculates accuracy by comparing predictions with true labels.
 * Returns the average validation loss and top-1 and top-5 accuracies.


* These functions should be called in a loop for multiple epochs. Your final model should converge on the training data, so make sure to use an adequate number of epochs for proper training. You need to store and print the loss, top-1 and top-5 accuracies values for both training and validation after each epoch to track model performance.

In [ ]:
def train(model, train_loader, optimizer, loss_function, device):
    # get the model to training mode
    model.train()

    total_loss = 0.0
    total_top1 = 0.0
    total_top5 = 0.0
    total_samples = 0

    for batch in train_loader:

        #####################################################
        # @TODO: get images and their labels from the batch
        pass
        #####################################################


        #####################################################
        # @TODO: pass your tensor to your device (CPU or GPU)
        pass
        #####################################################


        #####################################################
        # @TODO: Implement forward pass
        # HINT: Compute predictions and loss
        pass
        #####################################################

        total_loss += None # Modify this line

        #####################################################
        # @TODO: Implement backward pass and optimization step
        # HINT: Clear gradients, backpropagate, and update weights
        pass
        #####################################################

        #####################################################
        # @TODO: Update accuracy metrics and loss
        pass
        #####################################################

    avg_loss = total_loss / total_samples
    avg_top1 = 100.0 * total_top1 / total_samples
    avg_top5 = 100.0 * total_top5 / total_samples

    return avg_loss, avg_top1, avg_top5

def validate(model, val_loader, loss_function, device):
    # get the model to evaluation mode
    model.eval()

    running_loss = 0.0
    total_top1 = 0.0
    total_top5 = 0.0
    total_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            #####################################################
            # @TODO: extract images and label from batch
            pass
            #####################################################

            # TODO: pass your tensor to your device(CPU/GPU)
            pass
            #####################################################

            # TODO: Implement validation forward pass
            # HINT: Compute predictions and loss
            pass
            #####################################################

            #####################################################
            # TODO: Calculate accuracies
            # HINT: Compare predictions with labels
            pass
            #####################################################

    avg_loss = running_loss / total_samples
    avg_top1 = 100.0 * total_top1 / total_samples
    avg_top5 = 100.0 * total_top5 / total_samples

    return avg_loss, avg_top1, avg_top5


#####################################################
# @TODO: Call train and validate functions in a loop for multiple epochs
# @TODO: Store loss value and accuracy of training and validation data for each epoch.
#####################################################

### **2.6 Plot Loss Curves and Test Accuracies (5 Points)**
In this task, you should plot two different graphs:

1.   Training and validation losses versus epoch in a single graph.
2.   Training and validation accuracies (both top-1 and top-5) versus epoch in a single graph.

You should also report test accuracies (top-1 and top-5).

In [ ]:
import matplotlib.pyplot as plt
import tqdm

def plot_loss(train_losses, val_losses):
    # train_losses:    List to store training losses for each epoch
    # val_losses:      List to store validation losses for each epoch

    #####################################################
    # @TODO: plot the training and validation loss
    pass
    #####################################################

    plt.show()

def plot_accuries(train_top1, train_top5, val_top1, val_top5):
    # train_top1: List to store training top-1 accuries for each epoch
    # train_top5: List to store training top-5 accuries for each epoch
    # val_top1:   List to store val top-1 accuries for each epoch
    # val_top5:   List to store val top-5 accuries for each epoch

    #####################################################
    # @TODO: plot the training and validation accuries
    pass
    #####################################################
    plt.show()

def report_accuracy(test_loader, model):
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for batch in test_loader:

            #####################################################
            # @TODO: get images and their labels from batch

            # @TODO: pass your tensor to your device( CPU/GPU)

            # @TODO: Implement validation forward pass
            # HINT: Compute predictions and loss

            # @TODO: Calculate accuracy
            # HINT: Compare predictions with labels
            pass
            #####################################################

    accuracy_top1 = (correct_top1 / total) * 100
    accuracy_top5 = (correct_top5 / total) * 100
    return accuracy_top1,accuracy_top5

# call plot_loss and compute_confusion matrix function with appropriate parameters
plot_loss(train_losses, val_losses) # @TODO: modify argument based on your code
plot_accuries(train_accuracies_top1, train_accuracies_top5, val_accuracies_top1,val_accuracies_top5) #TODO: modify argument based on your code

test_accuracy = report_accuracy(test_loader, model) # @TODO: modify argument based on your code
print(f"Test Accuracy top-1 and top-5: {test_accuracy[0]:.2f}% {test_accuracy[1]:.2f}%")


### **2.7 Change and Finetune Your Model (20 Points)**

* Add BatchNorm2d with learnable parameters after each Conv2D layer.

* Try different optimizers including SGD, Adam.

* Search for a better **learning rate** on the validation data.

* For each combination of model, optimizer and learning settings plot graphs mentioned in Section 2.7.

* For the best parameters, model and optimizer settings, report test accuricies.

* **Discuss your results** with respect to overfitting/underfitting and the impact of the hyper-parameters.

#### **2.7.1 Hyperparameter Optimization (5 Points)**
In this part, you should perform hyperparameter optimization using all possible combinations (grid-search) of the given learning rates, optimizer and BatchNorm2d settings.

First, you are expected re-implement your model in Section 2.2 using BatchNorm2D layer after each convlution layer.

In [ ]:
class CustomCNNwithBN(nn.Module):
    def __init__(self, norm_layer= None):
        super(CustomCNNwithBN, self).__init__()

        #####################################################
        # @TODO: Define layers of CNN implemented in Section 2.3.
        # You should use BatchNorm2D layer after each Conv2D layer.
        pass
        #####################################################

    def forward(self, x):

        #####################################################
        # @TODO: Implement the forward pass
        # HINT: Apply activation functions like ReLU where necessary
        pass
        #####################################################

        return x

# Create model instance
model_bn = CustomCNNCustomCNNwithBN()

# Print model summary
print(model_bn)

For each combination, you should retrain your model from scratch and store the loss and accuracies of validation data. **All remaining settings/parameters should be fixed, except for the learning rate, optimizer and BatchNorm2d usage.**

In [ ]:
learning_rates = [0.0001, 0.001]
optimizer_classes = [torch.optim.Adam, torch.optim.SGD]
model_list = [model, model_bn]

# @TODO: implement a nested for loop to make a grid search over the given
# learning_rates, models and optimizers. Do not forget to reinitialize the model
# to avoid weight sharing.

#### **2.7.2 Choosing best parameters  (5 Points)**
You should choose the parameters that give the highest top-1 accuracy on the validation data. Then, you should plot graph and report metrics mentioned in Section 2.7.




In [ ]:
# @TODO: Find the best parameters that give the highest accuracy on the validation data.


#### **2.7.3 Discuss your results (10 Points)**
In this section, you should discuss your results with respect to overfitting/underfitting and the impact of the hyper-parameters.

## **Task 3: Implementing a RNN via torch.autograd (30 Points)**
In this task, you should implement a Recurrent Neural Network (RNN) by manually unrolling an RNN and using PyTorch's `torch.autograd` to compute gradients explicitly.





### **3.1 Input Pre-processing**
In this part, you are expected to process an input string to estimate a given target. We use the string "Deep Learning" for a character-level prediction task. For example:


```
input:  D e e p   L e a r n i n
target: e e p   L e a r n i n g
```

Each character should be converted to a one-hot vector, and the model should predict the next character in the sequence.


In [ ]:
import torch

#######################################################
# 1. Define input text and build vocabulary
#######################################################

text = "Deep Learning"  # Input text for character-level modeling

# Create a sorted list of unique characters.
# 'chars' will contain all unique characters in the input text sorted in alphabetical order
chars = None # @TODO: modify this line

# Create a mapping from character to index (char2idx) and index to character (idx2char)
# 'char2idx' will map each unique character to a unique index, and 'idx2char' will reverse that mapping
char2idx = None # @TODO: modify this line
idx2char = None # @TODO: modify this line

#######################################################
# 2. Create input and target sequences
#######################################################

# Input: all characters except the last one
# This is the input sequence for the RNN model. The input sequence will be "Deep Learnin"
input_seq = None  # @TODO: modify this line

# Target: all characters except the first one
# The target sequence is the same as the input sequence, but shifted by one character.
# The target sequence will be "eep Learning"
target_seq = None  # @TODO: modify this line

#######################################################
# 3. Define model hyperparameters
#######################################################

V = len(chars)       # Size of character vocabulary. Number of unique characters in the text.
input_size = V       # One-hot input vector size. It's the same as vocab size, as each character is represented as one-hot.
H = 16              # Number of hidden units in the RNN. This defines the dimensionality of the hidden state.
output_size = V      # Output size. For each character in the sequence, we output a score for each possible character.
seq_len = len(input_seq)      # Length of input sequence (i.e., the number of time steps).
                              # This is the number of characters in the input sequence.


# At this point, we have all the necessary components to train an RNN:
# - A list of characters and their mappings to indices
# - The input sequence and corresponding target sequence
# - The hyperparameters that define the model's architecture

#######################################################
# 4. Convert characters to one-hot vectors
#######################################################

def one_hot(idx, size):
    """
    Convert index to one-hot encoded vector of given size.

    Args:
    idx (int): The index of the character in the vocabulary.
    size (int): The size of the one-hot encoded vector, typically equal to V.

    Returns:
    torch.Tensor: A one-hot encoded vector of size 'size' where the element at index 'idx' is 1, and all others are 0.
    """
    #####################################################
    # @TODO: Implement one-hot decoding
    #####################################################
    vec = None
    return vec

# Convert entire input sequence to one-hot encoded tensors
# The input sequence is a string, and each character needs to be converted to a one-hot vector using the 'one_hot' function.
# For each character in the input sequence, its index is retrieved from 'char2idx', and then the corresponding one-hot vector is generated.

inputs = None # @TODO: modify this line (Convert entire input sequence to one-hot encoded tensors)

# Convert target characters to integer labels (indices)
# The target sequence contains the next character for each input character, so each character needs to be mapped to an index.
# This is a typical setup where the model outputs indices, which will later be converted to actual characters.

targets = None # @TODO: modify this line (Convert target characters to integer labels (indices))

### **3.2 Implement a Training Pipeline (30 Points)**

In this part, you should implement the following RNN for an autoregressive character prediction problem with the vocabulary size (the number of possible characters) of ${V}$, the hidden state size (the number of hidden units) of ${H}$, and given the input-output pair $(\mathbf{x}=(\mathbf{x}_1, \mathbf{x}_2, ...), y)$ -- e.g., ($\mathbf{x}=($'D', 'e', 'e'$), y=$'p'):

\begin{align*}
\mathbf{h}_t &= \tanh(\mathbf{W}_{xh} \, \mathbf{x}_t + \mathbf{b}_{ih} + \mathbf{W}_{hh} \, \mathbf{h}_{t-1} + \mathbf{b}_{hh}), \\
\mathbf{s}_t &= \mathbf{W}_{hy} \, \mathbf{h}_t + \mathbf{b}_y, \\
\mathbf{\hat{y}}_t &= \text{softmax}(\mathbf{s}_t),
\end{align*}  
where

*      $\mathbf{x}_t$: one-hot encoded input at time step $t$ (size: $V$),
*      $\mathbf{h}_t$: hidden state at time step $t$ (size: $H$),
*      $\mathbf{s}_t$: output logits at time step $t$ (size: $V$),
*      $\mathbf{\hat{y}}_t$: predicted probabilities (size: $V$),
*      $\mathbf{W}_{xh}$: input-to-hidden weight matrix (size: $H \times V$),
*      $\mathbf{W}_{hh}$: hidden-to-hidden weight matrix (size: $H \times H$),
*      $\mathbf{b}_{xh}$: hidden bias (size: $H$),
*      $\mathbf{b}_{hh}$: hidden bias (size: $H$),
*      $\mathbf{W}_{hy}$: hidden-to-output weight matrix (size: $V \times H$),
*      $\mathbf{b}_y$: output bias (size: $V$).

Then, you should calculate **negative log likelihood loss (NLL)** between predicted probabilities ${\mathbf{\hat{y}}}$ and target $\mathbf{y}$:


$${L}_{\text{NLL}} = - \sum_{i=1}^{N} \sum_{c=1}^{V} \mathbf{y}_{ic} \log \mathbf{\hat{y}}_{ic}$$



In [ ]:
###########################################################################
# @TODO: Define W_xh, W_hh, b_xh, b_hh, W_hy and b_y as differentiable tensors.
#        Initialize all weights randomly using a standard normal distribution.
# HINT: Use the given V and H in coding cell of "Section 3.1 Input Pre-processing" to define them.

# Do not modify variables names!
W_xh = None # @TODO: modify this line
W_hh = None # @TODO: modify this line
b_xh = None # @TODO: modify this line
b_hh = None # @TODO: modify this line
W_hy = None # @TODO: modify this line
b_y = None  # @TODO: modify this line
###########################################################################


logits_list = []
h = torch.zeros(H)
# Go over each time step to update the hidden state first and then calculate
# the logits
for t in range(seq_len):
    ###########################################################################
    # @TODO: Fill in this part to calculate logits
    pass
    s_t = None # modify this line
    ###########################################################################
    logits_list.append(s_t)

logits = torch.stack(logits_list)
log_probs = F.log_softmax(logits, dim=1)
loss_manual = F.nll_loss(log_probs, targets)

### **3.3 Compute the gradient explicitly via torch.autograd**
In this part, you are expected compute gradient using torch.autograd explicitly for each parameter, i.e. gradients of loss value with respect to:

*   $\mathbf{W}_{xh}$
*   $\mathbf{W}_{hh}$
*   $\mathbf{b}_{xh}$
*   $\mathbf{b}_{hh}$
*   $\mathbf{W}_{hy}$
*   $\mathbf{b}_{y}$



In [ ]:
# Calculate gradient for each parameter, separately.
grad_W_xh =  None # @TODO: modify this line
grad_W_hh =  None # @TODO: modify this line
grad_b_xh =  None # @TODO: modify this line
grad_b_hh =  None # @TODO: modify this line
grad_W_hy =  None # @TODO: modify this line
grad_b_y =   None # @TODO: modify this line


# Now, concatenate the gradients
manual_grads = [grad_W_xh, grad_W_hh, grad_b_xh, grad_b_hh, grad_W_hy, grad_b_y]

### **3.4 Validate Implementation in Section 3.3**
This part is only to validate your implementation in Section 3.3 and see your grade.

**Do not modify/add any code here!**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn_cell = nn.RNNCell(input_size, H)
        self.hidden_size = H
        self.output_layer = nn.Linear(H, output_size)

        with torch.no_grad():
            self.rnn_cell.weight_ih.copy_(W_xh)  # manual weight W_xh matches weight_ih
            self.rnn_cell.weight_hh.copy_(W_hh)
            self.rnn_cell.bias_ih.copy_(b_xh)
            self.rnn_cell.bias_hh.copy_(b_hh)

            self.output_layer.weight.copy_(W_hy)
            self.output_layer.bias.copy_(b_y)

    def forward(self, x_seq):
        h = torch.zeros(self.hidden_size)
        logits = []
        for x in x_seq:
            h = self.rnn_cell(x, h)
            y = self.output_layer(h)
            logits.append(y)
        return torch.stack(logits)

grade = 0
model = SimpleRNN()

logits_ref = model(inputs)
log_probs_ref = F.log_softmax(logits_ref, dim=1)
loss_ref = F.nll_loss(log_probs_ref, targets)
loss_ref.backward()

# ----------------------------
# 5. Compare Gradients
# ----------------------------

param_names = ['W_xh', 'W_hh', 'b_xh', 'b_hh', 'W_hy', 'b_y']
model_grads = []

#get model gradient
for name, param in model.named_parameters():
    if param.grad is not None:
        model_grads.append(param.grad)

for name, g1, g2 in zip(param_names, manual_grads, model_grads):
    diff = (g1 - g2).abs().max()
    print(f"Δ {name}: max abs diff = {diff:.6e}")
    if diff > 1e-4:
        print("❗ Significant difference")
    else:
        grade += 5
print(f" Your grade is {grade}/30")